In [ ]:
!curl --request GET --url host.docker.internal:21434/api/tags

In [ ]:
from langchain_ollama import ChatOllama

### llm 
local_llm="deepseek-r1:1.5b-qwen-distill-fp16"
llm = ChatOllama(model=local_llm, temperature=0, base_url="http://host.docker.internal:21434")
llm_json = ChatOllama(model=local_llm, temperature=0, return_json=True, base_url="http://host.docker.internal:21434")

In [ ]:
msg = llm.invoke("What is the capital of France?")
msg.content

__________________

# GROQ test

In [ ]:
import os

from groq import Groq
from dotenv import load_dotenv
load_dotenv("../../groq.env")


In [ ]:
client = Groq(
    # This is the default and can be omitted
    api_key=os.getenv("GROQ_API_KEY"),
)

In [ ]:
model_name = os.getenv("GROQ_LLM")
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "you are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model=model_name,
    reasoning_format={"type": "groq"},
    temperature=0.6,
    max_completion_tokens=131072,
    top_p=0.95,
    stream=True,
    stop=None,
    
)

In [ ]:

for chunk in chat_completion:
    print(chunk.choices[0].delta.content or "", end="")

__________


# GROQ langchain test

In [1]:
import os
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from ollama_deep_researcher.prompts import (
    query_writer_instructions,
    summarizer_instructions,
    reflection_instructions,
    get_current_date,
)
from dotenv import load_dotenv
load_dotenv("../../groq.env")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
model_name = os.getenv("GROQ_LLM")

In [2]:
current_date = get_current_date()
formatted_prompt = query_writer_instructions.format(
        current_date=current_date, research_topic="fast language models"
    )

In [3]:
llm = ChatGroq(
    model=model_name,
    temperature=0,
    response_format={"type": "json_object"},
)
#llm_json_mode = llm.with_structured_output(method="json_mode")

/home/jens/.cache/pypoetry/virtualenvs/ollama-deep-researcher-AmCmqQEy-py3.12/lib/python3.12/site-packages/langchain_groq/chat_models.py:364: UserWarning: WARNING! response_format is not default parameter.
                    response_format was transferred to model_kwargs.
                    Please confirm that response_format is what you intended.
  warnings.warn(


In [ ]:
result = llm_json_mode.invoke(
    [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content=f"Generate a query for web search:"),
    ]
)

In [4]:
result2 = llm.invoke(
    [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content=f"Generate a query for web search:"),
    ]
)

In [5]:
print(result2.content)

{    "query": "recent advancements in efficient transformer-based language models 2024",
    "rationale": "This query focuses on the latest developments in making transformer models faster and more efficient, which is crucial for understanding current trends in language model optimization."
}


_____

# tavily test (Used for making the crawler search)

In [1]:
import os
from ollama_deep_researcher.utils import (
    deduplicate_and_format_sources,
    tavily_search,
    format_sources,
)
from dotenv import load_dotenv
load_dotenv("../../.env")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [2]:
search_results = tavily_search("what is the capital of France?", fetch_full_page=True, max_results=3)

In [4]:
search_results

{'query': 'what is the capital of France?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'What is the Capital of France? - WorldAtlas',
   'url': 'https://www.worldatlas.com/articles/what-is-the-capital-of-france.html',
   'content': 'Paris is the capital city of France. The city has an approximate area of 41 square miles with a population of 2,206,488 people as of 2018. Instead, the name Paris is derived from the city’s initial inhabitants who were part of the Celtic Parisii tribe. Sometimes, the city is called the City of Light for two reasons; it was among the first cities to adopt gas for lighting the streets and its role during the Age of Enlightenment. Administration As the capital city of France, the city plays host to the national government of France.',
   'score': 0.9203972,
   'raw_content': 'What is the Capital of France? - WorldAtlas\nWhat is the Capital of France?\n\nTourism has always been a major source of income for Paris.\nParis

In [5]:
search_str = deduplicate_and_format_sources(
            search_results,
            max_tokens_per_source=10000000,
            fetch_full_page=True,
        )

In [7]:
print(search_str)

Sources:

Source: What is the Capital of France? - WorldAtlas
===
URL: https://www.worldatlas.com/articles/what-is-the-capital-of-france.html
===
Most relevant content from source: Paris is the capital city of France. The city has an approximate area of 41 square miles with a population of 2,206,488 people as of 2018. Instead, the name Paris is derived from the city’s initial inhabitants who were part of the Celtic Parisii tribe. Sometimes, the city is called the City of Light for two reasons; it was among the first cities to adopt gas for lighting the streets and its role during the Age of Enlightenment. Administration As the capital city of France, the city plays host to the national government of France.
===
Full source content limited to 10000000 tokens: What is the Capital of France? - WorldAtlas
What is the Capital of France?

Tourism has always been a major source of income for Paris.
Paris is the capital city of France. The city has an approximate area of 41 square miles with a